In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('immo_cat.csv')
data = data.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)
cols = ['newlyConst', 'balcony', 'hasKitchen','lift', 'garden','cellar']
data[cols] = data[cols].replace({True:1,False:0})
pd.set_option('display.max_columns', None)
data.head(1)

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date,Euro/m2,price_class,typeOfFlat_cat,heatingType_cat,firingTypes_cat,regio1_cat,condition_cat,interiorQual_cat,petsAllowed_cat,noRoomsRange_cat,energyEfficiencyClass_cat
0,Nordrhein_Westfalen,245.0,central_heating,ONE_YEAR_FREE,0.0,0,0,6,4.62,10.0,840.0,1965.0,96107057,1.0,oil,0,Nordrhein_Westfalen,1,2.0,595.0,244,86.0,Dortmund,well_kept,normal,unknown,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,0,4,ground_floor,44269,4.0,181.4,1.0,3.0,4,1,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,77.111,unknown,2009.1,89.11,0.21111,May19,6.92,4,0.4,0.55,0.45,0.51,0.57,0.3,0.95,4.0,0.61


In [3]:
import re
def clean_up(s):
    """
    Cleans up numbers, URLs, and special characters from a string.

    Args:
        s: The string to be cleaned up.

    Returns:
        A string that has been cleaned up.
    """
    s = s.lower()
    s = s.replace('@', '')
    s = s.replace('#', '')
    # s = s.replace('http://', '')
    s = re.sub(r'http\S+', '', s)
    # s = re.sub(r'www\S+', '', s)
    s = re.sub(r'\d+', '', s)
    s = s.replace('[', '')
    s = s.replace('(', '')
    s = s.replace(')', '')
    s = s.replace(']', '')
    s = s.replace("'", ' ')
    s = s.replace("-", ' ')
    
    return s

In [4]:
from nltk.tokenize import word_tokenize
def tokenize(s):
    """
    Tokenize a string.

    Args:
        s: String to be tokenized.

    Returns:
        A list of words as the result of tokenization.
    """
    tokens = word_tokenize(s)
    words = [word.lower() for word in tokens if word.isalnum()]

    return words

In [5]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.corpus import wordnet

def stem_and_lemmatize(l):
    por_ste = PorterStemmer()
    stemmer = [por_ste.stem(w) for w in l]
    lemmatizer = WordNetLemmatizer() 
    lemmatizing = [lemmatizer.lemmatize(word) for word in l]
    
    return stemmer, lemmatizing

In [6]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('german'))
def remove_stopwords(l):
    """
    Remove German stopwords from a list of strings.

    Args:
        l: A list of strings.

    Returns:
        A list of strings after stop words are removed.
    """
    cleaned_list = []
    for text in l:
        words = text.split()
        filtered_words = [w for w in words if w.lower() not in stop_words]
        cleaned_list.append(' '.join(filtered_words))
        
    return cleaned_list

In [7]:
#test
l = ['Nordkorea hat nach Angaben des südkoreanischen Militärs eine atomwaffenfähige Rakete mit einer Reichweite von möglicherweise tausenden Kilometern in Richtung offenes Meer abgefeuert. Der Start sei am Donnerstagmorgen (Ortszeit) in der Nähe der nordkoreanischen Hauptstadt Pjöngjang erfolgt, teilte der Generalstab in Seoul mit. Die Rakete sei dann in Richtung Japanisches Meer (koreanisch: Ostmeer) geflogen.']
cleaned_list = remove_stopwords(l)
print(cleaned_list)

['Nordkorea Angaben südkoreanischen Militärs atomwaffenfähige Rakete Reichweite möglicherweise tausenden Kilometern Richtung offenes Meer abgefeuert. Start sei Donnerstagmorgen (Ortszeit) Nähe nordkoreanischen Hauptstadt Pjöngjang erfolgt, teilte Generalstab Seoul mit. Rakete sei Richtung Japanisches Meer (koreanisch: Ostmeer) geflogen.']


In [8]:
data['cleaned_up'] = data['description'].apply(clean_up)
data['tokenized'] = data['cleaned_up'].apply(tokenize)
data['stemmed'], data['lemmatized'] = zip(*data['tokenized'].apply(stem_and_lemmatize))
data['text_processed'] = data['tokenized'].apply(remove_stopwords)
data.head()

,regio1,serviceCharge,heatingType,telekomTvOffer,telekomHybridUploadSpeed,newlyConst,balcony,picturecount,pricetrend,telekomUploadSpeed,totalRent,yearConstructed,scoutId,noParkSpaces,firingTypes,hasKitchen,geo_bln,cellar,yearConstructedRange,baseRent,houseNumber,livingSpace,geo_krs,condition,interiorQual,petsAllowed,street,streetPlain,lift,baseRentRange,typeOfFlat,geo_plz,noRooms,thermalChar,floor,numberOfFloors,noRoomsRange,garden,livingSpaceRange,regio2,regio3,description,facilities,heatingCosts,energyEfficiencyClass,lastRefurbish,electricityBasePrice,electricityKwhPrice,date,Euro/m2,price_class,typeOfFlat_cat,heatingType_cat,firingTypes_cat,regio1_cat,condition_cat,interiorQual_cat,petsAllowed_cat,noRoomsRange_cat,energyEfficiencyClass_cat,cleaned_up,tokenized,stemmed,lemmatized,text_processed
0,Nordrhein_Westfalen,245.00,central_heating,ONE_YEAR_FREE,0.0,0,0,6,4.62,10.0,840.00,1965.0,96107057,1.0,oil,0,Nordrhein_Westfalen,1,2.0,595.0,244,86.00,Dortmund,well_kept,normal,unknown,Sch&uuml;ruferstra&szlig;e,Schüruferstraße,0,4,ground_floor,44269,4.0,181.40,1.0,3.0,4,1,4,Dortmund,Schüren,Die ebenerdig zu erreichende Erdgeschosswohnun...,Die Wohnung ist mit Laminat ausgelegt. Das Bad...,77.111,unknown,2009.1,89.11,0.21111,May19,6.92,4,0.4,0.55,0.45,0.51,0.57,0.30,0.95,4.0,0.61,die ebenerdig zu erreichende erdgeschosswohnun...,"[die, ebenerdig, zu, erreichende, erdgeschossw...","[die, ebenerdig, zu, erreichend, erdgeschosswo...","[die, ebenerdig, zu, erreichende, erdgeschossw...","[, ebenerdig, , erreichende, erdgeschosswohnun..."
1,Rheinland_Pfalz,134.00,self_contained_central_heating,ONE_YEAR_FREE,0.0,0,1,8,3.47,10.0,814.11,1871.0,111378734,2.0,gas,0,Rheinland_Pfalz,0,1.0,800.0,1,89.00,Rhein_Pfalz_Kreis,refurbished,normal,no,no_information,unknown,0,5,ground_floor,67459,3.0,114.11,2.1,3.1,3,0,4,Rhein_Pfalz_Kreis,Böhl_Iggelheim,Alles neu macht der Mai – so kann es auch für ...,unknown,77.111,unknown,2019.0,89.11,0.21111,May19,8.99,6,0.4,0.55,0.45,0.58,0.52,0.30,1.00,3.0,0.61,alles neu macht der mai – so kann es auch für ...,"[alles, neu, macht, der, mai, so, kann, es, au...","[all, neu, macht, der, mai, so, kann, es, auch...","[alles, neu, macht, der, mai, so, kann, e, auc...","[, neu, macht, , mai, , , , , , , , , genießen..."
2,Sachsen,255.00,floor_heating,ONE_YEAR_FREE,10.0,1,1,8,2.72,2.4,1300.00,2019.0,113147523,1.0,unknown,0,Sachsen,1,9.0,965.0,4,83.80,Dresden,first_time_use,sophisticated,unknown,Turnerweg,Turnerweg,1,6,apartment,1097,3.0,114.11,3.0,4.0,3,0,4,Dresden,Äußere_Neustadt_Antonstadt,Der Neubau entsteht im Herzen der Dresdner Neu...,"* 9 m² Balkon\n* Bad mit bodengleicher Dusche,...",77.111,unknown,2009.1,89.11,0.21111,Oct19,11.52,8,0.4,1.00,0.45,0.38,1.00,0.56,0.95,3.0,0.61,der neubau entsteht im herzen der dresdner neu...,"[der, neubau, entsteht, im, herzen, der, dresd...","[der, neubau, entsteht, im, herzen, der, dresd...","[der, neubau, entsteht, im, herzen, der, dresd...","[, neubau, entsteht, , herzen, , dresdner, neu..."
3,Sachsen,58.15,district_heating,ONE_YEAR_FREE,0.0,0,1,9,1.53,40.0,814.11,1964.0,108890903,0.0,district_heating,0,Sachsen,0,2.0,343.0,35,58.15,Mittelsachsen_Kreis,unknown,unknown,unknown,Gl&uuml;ck-Auf-Stra&szlig;e,Glück-Auf-Straße,0,2,other,9599,3.0,86.00,3.0,3.1,3,0,2,Mittelsachsen_Kreis,Freiberg,Abseits von Lärm und Abgasen in Ihre neue Wohn...,unknown,87.230,unknown,2009.1,89.11,0.21111,May19,5.90,3,0.4,0.55,0.45,0.38,0.61,0.37,0.95,3.0,0.61,abseits von lärm und abgasen in ihre neue wohn...,"[abseits, von, lärm, und, abgasen, in, ihre, n...","[abseit, von, lärm, und, abgasen, in, ihr, neu...","[abseits, von, lärm, und, abgasen, in, ihre, n...","[abseits, , lärm, , abgasen, , , neue, wohnung..."
4,Bremen,138.00,self_contained_central_heating,unknown,0.0,0,1,19,2.46,40.0,903.00,1950.0,114751222,0.0,gas,0,Bremen,0,1.0,765.0,10,84.97,Bremen,refurbished,unknown,unknown,Hermann-Henrich-Meier-Allee,Hermann-Henrich-Meier-Allee,0,5,apartment,28213,3.0,188.90,1.0,3.1,3,0,4,Bremen,Neu_Schwachhause

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk import FreqDist
import numpy as np

words = [' '.join(words) for words in data['text_processed']]

vector = CountVectorizer(max_features=500)

X = vector.fit_transform(words)

count = np.asarray(X.sum(axis=0)).ravel()

feature_names = vector.get_feature_names()

dict = dict(zip(feature_names, count))

dist = FreqDist(dict)

dist.most_common(100)
#list the 50 most common words in data['description'] (without stop words)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


[('wohnung', 310274),
 ('befindet', 100327),
 ('zimmer', 91854),
 ('küche', 80223),
 ('sowie', 77526),
 ('balkon', 67956),
 ('verfügt', 56630),
 ('bad', 55901),
 ('schlafzimmer', 50899),
 ('wurde', 50776),
 ('ca', 48974),
 ('haus', 47277),
 ('m²', 46874),
 ('wohnzimmer', 45393),
 ('ausgestattet', 44747),
 ('wohnungen', 43678),
 ('liegt', 42752),
 ('verfügung', 41551),
 ('einbauküche', 41032),
 ('objekt', 40147),
 ('badezimmer', 38403),
 ('bietet', 36809),
 ('dusche', 36118),
 ('zwei', 35904),
 ('wc', 35096),
 ('wohn', 34411),
 ('ab', 33280),
 ('mehrfamilienhaus', 32264),
 ('kellerraum', 32137),
 ('fenster', 31780),
 ('lage', 30204),
 ('große', 29325),
 ('stellplatz', 29122),
 ('schöne', 28861),
 ('flur', 27983),
 ('vorhanden', 27218),
 ('helle', 26809),
 ('gepflegten', 26585),
 ('steht', 26066),
 ('stehen', 25467),
 ('gehört', 25112),
 ('direkt', 24598),
 ('handelt', 24258),
 ('erreichen', 24233),
 ('obergeschoss', 24203),
 ('großen', 23573),
 ('badewanne', 23216),
 ('platz', 22855),
 